In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

import requests
from bs4 import BeautifulSoup
# from sklearn.Logit_model_reloaded import score
# data_test = pd.read_csv("test_notes.csv", sep=',')

In [2]:
# nice, psg,om, lyon, monaco, lille,rennes, mtp
team_list_av = []
x_av = []
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

page = "https://www.transfermarkt.fr/ligue-1/startseite/wettbewerb/FR1"

pageTree = requests.get(page, headers=headers)
soup = BeautifulSoup(pageTree.content, 'html.parser')
body = soup.html.find('div',{'class':'large-8 columns'}).find_all('div',{'class':'box'})[1].find('div',{'class':'responsive-table'}).div.table.tbody.find_all('tr')
ixo2 = []
for g in body:

    team_list_av.append(g.find('td',{'hauptlink no-border-links show-for-small show-for-pad'}).a.text)
    ixo = g.find_all('td',{'class':'rechts show-for-small show-for-pad nowrap'})[1].text.replace(' mio. €','').replace(',','.')
    

    if 'mrd' in ixo:
        ixo2.append(float(ixo.replace(' mrd. €',''))*1000)
    else:
        ixo2.append(ixo)
        
    x_av.append(ixo2)

In [3]:
value_model_av = pd.DataFrame({'Team':team_list_av,'Value':x_av[0]})
value_model_av.to_csv('value_model_av.csv',index=False)

In [4]:
value_model_af = pd.read_csv('value_model_av.csv',sep=',')

In [18]:
x = value_model_af['Value']
list_y = []
for i in x:
    list_y.append(0.014208735689186128*i + -2.5281681138672534)

In [19]:
value_model = pd.DataFrame({'Team':value_model_af['Team'],'Value':value_model_af['Value'],'trend':list_y})

In [20]:
value_model

,Team,Value,trend
0,Paris SG,993.25,11.584659
1,AS Monaco,385.85,2.954273
2,Ol. Lyonnais,326.15,2.106011
3,LOSC Lille,271.10,1.323820
4,Rennes,250.15,1.026147
5,Marseille,249.90,1.022595
6,OGC Nice,215.25,0.530262
7,Montpellier,116.30,-0.875692
8,Stade Reims,93.98,-1.192831
9,Saint-Étienne,93.70,-1.196810


In [21]:
0.013651660777911766*2000+ -2.399976874866872

24.903344680956657

In [22]:
# Récupération du modèle de Régression Logistique 'lr'
with open('model_mhsc.pkl', 'rb') as f:
    my_unpickler = pickle.Unpickler(f)
    model_logit = my_unpickler.load()

    y_pred = model_logit.predict_proba(value_model[['trend']].values) 


In [23]:
for value in y_pred[0]:
    print(value)

0.00023172884917310732
0.9997682711508269


In [24]:
predictions = [round(value) for value in y_pred[0]]
# print(ytest)
# precision = model_logit.score(value_model[['Value','trend']],predictions)
# print(np.around(precision,decimals=2)*100,'%')

In [25]:
value_model

,Team,Value,trend
0,Paris SG,993.25,11.584659
1,AS Monaco,385.85,2.954273
2,Ol. Lyonnais,326.15,2.106011
3,LOSC Lille,271.10,1.323820
4,Rennes,250.15,1.026147
5,Marseille,249.90,1.022595
6,OGC Nice,215.25,0.530262
7,Montpellier,116.30,-0.875692
8,Stade Reims,93.98,-1.192831
9,Saint-Étienne,93.70,-1.196810


In [26]:
predict = []
prt_predict = []
for i in y_pred:
    prt_predict.append(i[1])
value_model['pourcentage'] = prt_predict

count = 0
proba = 5
proba_list = []
for k,i in zip(value_model.sort_values(by='pourcentage',ascending=False)['Team'],value_model.sort_values(by='pourcentage',ascending=False)['pourcentage']):
    proba_list.append(np.around(i*(proba/5),decimals=3))
    print(i,np.around(i*(proba/5),decimals=3),count)
    count = count + 1
    if proba < 3:
        if count == 7:
            proba = proba - 1
        else:
            continue
    else:
        proba = proba - 1
        
    
value_model['pourcentage'] = proba_list
        
for i in list(value_model['pourcentage']):
    if i < 0.5:
        predict.append(False)
    elif i > 0.5:
        predict.append(True)
    
#print(len(predict))
#print(len(y_pred[1]))
value_model['predict'] = predict


0.9997682711508269 1.0 0
0.8387495950959165 0.671 1
0.7287688934203304 0.437 2
0.5936966355236154 0.237 3
0.5367985711925478 0.215 4
0.5361106976635436 0.214 5
0.44060662438756987 0.176 6
0.20857077242265087 0.042 7
0.17072067654735953 0.034 8
0.17028250392690422 0.034 9
0.16148110066119825 0.032 10
0.16036058372564238 0.032 11
0.15543229468139222 0.031 12
0.15492459427703834 0.031 13
0.14760993172833287 0.03 14
0.14436818041319943 0.029 15
0.13564437268986063 0.027 16
0.11371663544907086 0.023 17
0.10287923098496442 0.021 18
0.08925586949069061 0.018 19


In [27]:
print('Prédictions :', value_model['Team'][0],' : ', np.around(y_pred[0][1],decimals=2),' %')
print('Prédictions :', value_model['Team'][1],' : ', np.around(y_pred[1][1],decimals=2),' %')
print('Prédictions :', value_model['Team'][2],' : ', np.around(y_pred[2][1],decimals=2),' %')
print('Prédictions :', value_model['Team'][3],' : ', np.around(y_pred[3][1],decimals=2),' %')
print('Prédictions :', value_model['Team'][4],' : ', np.around(y_pred[4][1],decimals=2),' %')

Prédictions : Paris SG  :  1.0  %
Prédictions : AS Monaco  :  0.84  %
Prédictions : Ol. Lyonnais  :  0.73  %
Prédictions : LOSC Lille  :  0.59  %
Prédictions : Rennes  :  0.54  %


In [28]:
# for i,e in zip(value_model['Team'],value_model['pourcentage']):
#     print(i,e)
# MHSC 0.29
# PSG 1.0
# OM 0.89
# OL 0.97
# ASM 0.99
# LOSC 0.91
# NICE 0.77
# SRFC 0.83 

In [29]:
value_model

,Team,Value,trend,pourcentage,predict
0,Paris SG,993.25,11.584659,1.000,True
1,AS Monaco,385.85,2.954273,0.671,True
2,Ol. Lyonnais,326.15,2.106011,0.437,False
3,LOSC Lille,271.10,1.323820,0.237,False
4,Rennes,250.15,1.026147,0.215,False
5,Marseille,249.90,1.022595,0.214,False
6,OGC Nice,215.25,0.530262,0.176,False
7,Montpellier,116.30,-0.875692,0.042,False
8,Stade Reims,93.98,-1.192831,0.034,False
9,Saint-Étienne,93.70,-1.196810,0.034,False


In [30]:
value_model.to_csv('model_result.csv',index=False)